# Предварительный этап.

### Шаг 1. Подготовить FP_SNPs_10k_GB38_twoAllelsFormat.tsv файл.

1. Скачиваем и извлекаем из архива файл **FP_SNPs.txt** по схеме, представленной в пояснительной части тестового задания.
2. Создаём шаблон финального файла **FP_SNPs_10k_GB38_twoAllelsFormat.tsv** и добавляем в него правильные заголовки:

In [ ]:
echo -e "#CHROM\tPOS\tID\tallele1\tallele2" > FP_SNPs_10k_GB38_twoAllelsFormat.tsv

3. Преобразуем оригинальный FP_SNPs.txt файл и записываем скорректированные данные в финальный файл:

In [ ]:
awk '{print $2, $4, $1, $5, $6}' FP_SNPs.txt | awk '{print $1="chr"$1, $2, $3="rs"$3, $4, $5}' | head -n -1000 | sed '1d' | tr ' ' '\t' >> FP_SNPs_10k_GB38_twoAllelsFormat.tsv 
# select desired columns -> add prefixes (chr, rs) -> remove last 1000 rows (chr23) -> remove 1st line (column names) -> replace space-delimination to TABs ->> write modified table into FP_SNPs_10k_GB38_twoAllelsFormat.tsv 


### Шаг  2. Подготовить отдельные референсы для каждой хромосомы.

In [ ]:
 for i in (1..22, X, Y, M); do
	go to GRCh38.d1.vd1.fa
	select lines related to header (chr$i) and up to the next chromosome
	write lines into a new file 'chr$i.fai'
	do indexing of 'chr$i.fai'
 done

# Основной этап.

Необходимо выяснить, какой из двух аллелей является референсным, а какой – альтернативным.

Аргументы командной строки:

In [ ]:
InputTable # FP_SNPs_10k_GB38_twoAllelsFormat.tsv 
InputReference # Path to folder with references
OutputTable # output file name

#### Принцип работы скрипта:

Проверить аргументы:

In [ ]:
{
	arg is not empty
	arg has the right extention
	arg file structure is correct
}

Основная часть (после проверок аргументов командной строки) # explain what happens here

In [ ]:
{
 for i in (1..22); do
	str ChromosomeNumber = 'chr' + $i # make a string like 'chr1'
	FP_SNPs_$ChromosomeNumber.tsv # make a temporary table 'FP_SNPs_$ChromosomeNumber.tsv' for one chromosome
	FP_SNPs_$ChromosomeNumber.tsv # copy rows related to $ChromosomeNumber from 'FP_SNPs_10k_GB38_twoAllelsFormat.tsv' to 'FP_SNPs_$ChromosomeNumber.tsv'
	int RowsNumber = count 'number of rows' in FP_SNPs_$ChromosomeNumber.tsv
	int ErrorRowsNumber = 0 # используется для подсчёта позиций на хромосоме $ChromosomeNumber, где не удалось определить референсный аллель
    str RefGenome = .../$ChromosomeNumber.fai # go to $InputReference path and select referehce, related to $ChromosomeNumber

	for line in (1..$RowsNumber); do
	{
		int POS = ... # select POS on $line from FP_SNPs_$ChromosomeNumber.tsv
		str Allele1 = ... # select allele1 on $line from FP_SNPs_$ChromosomeNumber.tsv
		str Allele2 = ... # select allele2 on $line from FP_SNPs_$ChromosomeNumber.tsv
		str ID = ... # select ID on $line from FP_SNPs_$ChromosomeNumber.tsv

       !!! str RefNucl = ... # select nucleotide on position $POS in reference genome
		
		# Для каждой позиции $POS из FP_SNPs_$ChromosomeNumber.tsv
        # последовательно сравниваем нуклеотид на этой позиции в референсе RefNucl с allele1 и allele2
		
		if RefNucl == Allele1 
		{
		str REF == Allele1 
		str ALT == Allele2
		Записываем в новую таблицу строку, содержащую $ChromosomeNumber, $POS, $ID, $REF, $ALT
		}
		else if RefNucl == Allele2
		{
		str REF == Allele2
		str ALT == Allele1
		Записываем в новую таблицу строку, содержащую $ChromosomeNumber, $POS, $ID, $REF, $ALT
		}
		else
		{
		Add to log: "ERROR: reference nucleotide on the $ChromosomeNumber , position $POS differs from the input table ($InputTable)"
        ErrorRowsNumber += 1    
		}
		
		
		
		
	}
	
	
}

#### Сообщение, выдаваемое скриптом для описания себя и своих аргументов по запросу (-h, --help)